In [1]:
#Importing necessary Packages

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sb
from networkx.drawing import *
%matplotlib inline

In [3]:
#Importing data.Set low_memory=False to enable importing large files.

postsDF = pd.read_csv('/home/cloakedprat/csv/postTags.csv', names=['Id','Tags'],low_memory = False)

In [4]:
tagsDF = pd.read_csv('/home/cloakedprat/csv/Tags.csv', names = ['Id','TagName','Count','ExcerptPostId','WikiPostId'],low_memory = False)


In [5]:
#Sort Value

tagsDF.sort_values('Count',ascending = False,inplace = True)
tagsdf=tagsDF[:200]
tagsdf

,Id,TagName,Count,ExcerptPostId,WikiPostId
2,3,javascript,1339816,3624960,3607052
11,17,java,1223220,3624966,3607018
6,9,c#,1068888,3624962,3607007
4,5,php,1047779,3624936,3607050
700,1386,android,960996,3625001,3607484
420,820,jquery,822436,3625262,3607053
10,16,python,715171,3624965,3607014
1,2,html,632135,3673183,3673182
7,10,c++,502132,3624963,3606997
19239,58338,ios,495880,4536664,4536663


In [6]:
#Type of Networkx graph.

tagGraph = nx.MultiGraph()

In [14]:
#Generate a map of co-occurences.Takes a while to complete.Read a book.
#Only for the first time ever.

for Tag in tagsdf['TagName']:
    for Tag1 in tagsdf['TagName']:
        count = 0       
        if Tag!=Tag1:
            for Tag2 in postsDF['Tags']:
                if(Tag in Tag2 and Tag1 in Tag2 ):
                    count+=1
            tagGraph.add_edge(Tag,Tag1, weight = count)
    
            

In [57]:
#Convert Networkx mapping to Pandas DataFrame and save it as csv. You won't want to re-run the above cell again.

networkDF = nx.to_pandas_dataframe(tagGraph)
networkDF.to_csv('/home/cloakedprat/csv/network.csv')

In [49]:
#Check for any nodes that connect to themselves.
tagGraph.nodes_with_selfloops()


[]

In [19]:
#Convert to a graphml for Gephi.
nx.write_graphml(tagGraph,'/home/cloakedprat/csv/so.graphml')


In [ ]:
#Importing above calculated co-occurence map(Cuz..I did this a while after the above part.)
network=pd.read_csv('/home/cloakedprat/csv/network.csv')

In [384]:
#Divide your selected number of Tags into groups for easy color assignment. This is just a random arrangement.

group = list(network.index[:20])
group1= list(network.index[:40])
group2= list(network.index[:60])
group3= list(network.index[:80])
group4= list(network.index[:100])
group5= list(network.index[:120])

groups=[group,group1,group2,group3,group4,group5]

In [398]:
#Create pairs of (Tag,Tag) for Occurence heatmap. Alphas is a color-damping value, colors are assigned to every value with
#same group. Assign lightgrey to all pairs not in one group. Take one dict from Category10. 

x = []
y = []
value = []
colormap = []
color = []
alphas=[]
from bokeh.palettes import Category10
colormap=Category10[6]
for i in network.index[:120]:
    for j in network.index[:120]:
        x.append(i)
        y.append(j)
        value.append(network.get_value(i,j))
        
        alphas.append(min(network.get_value(i,j)/10000.0, 0.9) + 0.1)
        
        for k in range(len(groups)):
            if i and j in   groups[k]:color.append(colormap[k])
            elif k==5:color.append('lightgrey')
        
    

In [401]:
#Import Bokeh packages, assign data_source.

from bokeh.plotting import figure,show,output_file
from bokeh.models import HoverTool,ColumnDataSource 


source = ColumnDataSource(data=dict(
    xname=x,
    yname=y,
    colors = color,
    alphas = alphas,
    value=value
))

/home/cloakedprat/anaconda3/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('alphas', 14400), ('colors', 50400), ('value', 14400), ('xname', 14400), ('yname', 14400)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


In [411]:
#Configure plot features.

p = figure(title="Tag Co-occurence", tools="hover,save",
           x_axis_location="above", 
           x_range=network.index[:120].tolist(), y_range=list(reversed(network.index[:120].tolist())))

p.plot_width = 1000
p.plot_height = 1000
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "6pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/2

In [412]:
#Set each node to a square. Tooltips won't work without assigning tools in p above.

p.rect('xname', 'yname', 0.9, 0.9,color='colors',alpha = 'alphas', source=source, line_color=None,
       hover_line_color='black')

p.select_one(HoverTool).tooltips = [
    ('names', '@yname, @xname'),
    ('value', '@value'),
]

output_file("/home/cloakedprat/DL_Workx/taco.html", title="taco.py example")
show(p)